# Benchmarks for String Similarity Scoring Functions

Install the most commonly used Python packages for string similarity scoring. This includes JellyFish for Levenshtein and Levenshten-Damerau distance, RapidFuzz for Levenshtein distance, and BioPython for Needleman-Wunsh scores among others.

In [ ]:
!pip install rapidfuzz  # https://github.com/rapidfuzz/RapidFuzz
!pip install python-Levenshtein  # https://github.com/maxbachmann/python-Levenshtein
!pip install levenshtein # https://github.com/maxbachmann/Levenshtein
!pip install jellyfish # https://github.com/jamesturk/jellyfish/
!pip install editdistance # https://github.com/roy-ht/editdistance
!pip install distance # https://github.com/doukremt/distance
!pip install polyleven # https://github.com/fujimotos/polyleven
!pip install biopython # https://github.com/biopython/biopython
!pip install stringzilla # https://github.com/ashvardanian/stringzilla

## Levenshtein Distance Between Short English Words

We will be conducting benchmarks on a real-world dataset of English words. Let's download the dataset and load it into memory.

In [ ]:
!wget --no-clobber -O ../leipzig1M.txt https://introcs.cs.princeton.edu/python/42sort/leipzig1m.txt

In [ ]:
words = open("../leipzig1M.txt", "r").read().split()
words = tuple(words)
print(f"{len(words):,} words")

In [ ]:
import random

def checksum_distances(tokens, distance_function, n: int = 1000000):
    distances_sum = 0
    while n:
        a = random.choice(tokens)
        b = random.choice(tokens)
        distances_sum += distance_function(a, b)
        n -= 1
    return distances_sum

In [ ]:
import random

In [ ]:
proteins = [''.join(random.choice('ACGT') for _ in range(10_000)) for _ in range(1_000)]
print(f"{len(proteins):,} proteins")

In [ ]:
import stringzilla as sz

In [ ]:
%%timeit
checksum_distances(words, sz.edit_distance)

In [ ]:
%%timeit
checksum_distances(proteins, sz.edit_distance, 100)

In [ ]:
from rapidfuzz.distance import Levenshtein as rf

In [ ]:
%%timeit
checksum_distances(words, rf.distance)

In [ ]:
%%timeit
checksum_distances(proteins, rf.distance, 100)

In [ ]:
import editdistance as ed

In [ ]:
%%timeit
checksum_distances(words, ed.eval)

In [ ]:
import jellyfish as jf

In [ ]:
%%timeit
checksum_distances(words, jf.levenshtein_distance)

In [ ]:
import Levenshtein as le

In [ ]:
%%timeit
checksum_distances(words, le.distance)

## Needleman-Wunsch Alignment Scores Between Random Protein Sequences

For Needleman-Wunsh, let's generate some random protein sequences:

In [ ]:
from Bio import Align
from Bio.Align import substitution_matrices
aligner = Align.PairwiseAligner()
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
aligner.open_gap_score = 1
aligner.extend_gap_score = 1

In [ ]:
aligner.substitution_matrix

Let's convert the BLOSUM matrix into a dense form with 256x256 elements. This will allow us to use the matrix with the Needleman-Wunsh algorithm implemented in StringZilla.

In [ ]:
import numpy as np

subs_packed = np.array(aligner.substitution_matrix).astype(np.int8)
subs_reconstructed = np.zeros((256, 256), dtype=np.int8)

# Initialize all banned characters to a the largest possible penalty
subs_reconstructed.fill(127)
for packed_row, packed_row_aminoacid in enumerate(aligner.substitution_matrix.alphabet):
    for packed_column, packed_column_aminoacid in enumerate(aligner.substitution_matrix.alphabet):
        reconstructed_row = ord(packed_row_aminoacid)
        reconstructed_column = ord(packed_column_aminoacid)
        subs_reconstructed[reconstructed_row, reconstructed_column] = subs_packed[packed_row, packed_column]

(subs_reconstructed < 127).sum()

In [ ]:
aligner.score(proteins[0], proteins[1])

In [ ]:
sz.alignment_score(proteins[0], proteins[1], substitution_matrix=subs_reconstructed, gap_score=1)

In [ ]:
%%timeit
def sz_score(a, b): return sz.alignment_score(a, b, substitution_matrix=subs_reconstructed, gap_score=1)
checksum_distances(proteins, sz_score, 100)

In [ ]:
%%timeit
checksum_distances(proteins, aligner.score, 100)